In [ ]:
folderPath="/content/drive/MyDrive/JBC_HR_AI_ASSISTANT/PHP"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pandas
!pip install PyPDF2
!pip install openai
!pip install python-dateutil
!pip install tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.2 MB/s eta 0:00:00


In [ ]:
# /content/drive/MyDrive/JBC HR AI Assistant


# CV Analyzer for Google Drive - JBC HR AI Assistant
# Mount Google Drive, extract CV information from PDFs, and create an Excel output file

import os
import pandas as pd
import io
import PyPDF2
import openai
import re
import tempfile
import json
from datetime import datetime
import dateutil.parser
from dateutil.relativedelta import relativedelta
from google.colab import drive
from tqdm.notebook import tqdm

# Mount Google Drive
drive.mount('/content/drive')

# Set your OpenAI API key
openai_api_key = ""
openai.api_key = openai_api_key

# Define the root path
root_path = "/content/drive/MyDrive/JBC HR AI Assistant "

def extract_text_from_pdf(pdf_file):
    """Extract text content from a PDF file."""
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    text = ""
    for page_num in range(len(pdf_reader.pages)):
        text += pdf_reader.pages[page_num].extract_text()
    return text

def calculate_experience_duration(start_date_str):
    """Calculate duration between start date and current date in 'X year Y month' format."""
    try:
        # Parse the start date string
        if start_date_str == "Not found" or not start_date_str:
            return "Not found"

        # Try to parse the date with dateutil parser
        try:
            start_date = dateutil.parser.parse(start_date_str, fuzzy=True)
        except:
            # If parsing fails, try to extract month and year manually
            match = re.search(r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* (\d{4})',
                             start_date_str, re.IGNORECASE)
            if match:
                month_str = match.group(1)
                year_str = match.group(2)
                # Map abbreviated month to number
                month_map = {
                    'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4,
                    'may': 5, 'jun': 6, 'jul': 7, 'aug': 8,
                    'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
                }
                month = month_map.get(month_str.lower()[:3], 1)
                year = int(year_str)
                start_date = datetime(year, month, 1)
            else:
                return "Date format not recognized"

        # Calculate the difference between the start date and current date
        current_date = datetime.now()
        delta = relativedelta(current_date, start_date)

        # Format the result as "X year Y month"
        years = delta.years
        months = delta.months

        if years == 0:
            if months == 1:
                return f"{months} month"
            else:
                return f"{months} months"
        elif years == 1:
            if months == 0:
                return "1 year"
            elif months == 1:
                return "1 year 1 month"
            else:
                return f"1 year {months} months"
        else:
            if months == 0:
                return f"{years} years"
            elif months == 1:
                return f"{years} years 1 month"
            else:
                return f"{years} years {months} months"
    except Exception as e:
        return f"Error calculating duration: {str(e)}"

def extract_field(text, field_name):
    """Extract a specific field from text response when JSON parsing fails"""
    pattern = rf"{field_name}[:\s]+(.*?)(?:\n|$|,)"
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
        return match.group(1).strip()
    return "Not found"

def extract_cv_info(cv_text):
    """Use OpenAI API to extract structured information from CV text."""

    # Get current date for calculating work experience
    current_date = datetime.now()
    current_date_str = current_date.strftime("%Y %B")

    prompt = f"""
    Extract the following information from the CV text below.
    If you cannot find a particular piece of information, respond with "Not found" for that field.

    Information to extract:
    1. Name
    2. Last Education and university
    3. Number of total year experiences
    4. Present field of experience
    5. Overall expertise area
    6. Present organization designation
    7. Research experience (any research positions, publications, or projects)
    8. Achievements (awards, recognitions, significant accomplishments)
    9. Mobile number
    10. Email address
    11. Present organization name
    12. Working experience in present organization (start date in format 'Month YYYY', e.g. 'December 2022')

    Today is {current_date_str}.

    CV Text:
    {cv_text}

    Your response MUST be a valid JSON object with ONLY the following keys:
    {{
      "name": "extracted name",
      "last_education": "extracted education and university",
      "total_experience": "total number of experiences in all organizations",
      "present_field": "present field of experience",
      "overall_expertise_area": "areas of expertise or specialization",
      "present_organization_designation": "current job title or designation",
      "research_experience": "details of research experience if any",
      "achievements": "major achievements and awards if any",
      "mobile": "extracted mobile number",
      "email": "extracted email address",
      "present_organization_name": "name of current organization",
      "working_experience_in_present_organization": "start date in format 'Month YYYY'"
    }}

    Do not include any explanation, just return the JSON object.
    """

    try:
        # Use GPT-4o model for better extraction
        model = "gpt-4o"

        response = openai.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant that extracts structured information from CVs. Return only valid JSON."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3  # Lower temperature for more consistent results
        )

        # Extract and parse the JSON response
        result = response.choices[0].message.content

        # Try to find JSON in the response
        json_match = re.search(r'(\{[\s\S]*\})', result, re.DOTALL)
        if json_match:
            json_str = json_match.group(1)
            try:
                parsed_result = json.loads(json_str)

                # Calculate work experience duration
                start_date = parsed_result.get("working_experience_in_present_organization", "Not found")
                parsed_result["working_experience_in_year_in_present_organization"] = calculate_experience_duration(start_date)

                return parsed_result
            except json.JSONDecodeError:
                print(f"Failed to parse JSON from response. Attempting alternate extraction.")

        # If extraction failed, try to create a structured response manually
        try:
            # Create a standard response manually
            start_date = extract_field(result, "working_experience_in_present_organization")
            experience_duration = calculate_experience_duration(start_date)

            # Return in the specified order
            return {
                "name": extract_field(result, "name"),
                "last_education": extract_field(result, "last_education"),
                "overall_expertise_area": extract_field(result, "overall_expertise_area"),
                "present_organization_name": extract_field(result, "present_organization_name"),
                "present_organization_designation": extract_field(result, "present_organization_designation"),
                "working_experience_in_present_organization": start_date,
                "working_experience_in_year_in_present_organization": experience_duration,
                "total_experience": extract_field(result, "total_experience"),
                "present_field": extract_field(result, "present_field"),
                "research_experience": extract_field(result, "research_experience"),
                "achievements": extract_field(result, "achievements"),
                "mobile": extract_field(result, "mobile"),
                "email": extract_field(result, "email")
            }
        except Exception as e:
            print(f"Error creating structured response: {str(e)}")
            # Last resort, try direct JSON parsing
            parsed_result = json.loads(result)

            # Calculate work experience duration
            start_date = parsed_result.get("working_experience_in_present_organization", "Not found")
            parsed_result["working_experience_in_year_in_present_organization"] = calculate_experience_duration(start_date)

            return parsed_result

    except Exception as e:
        print(f"Error extracting information: {str(e)}")
        return {
            "name": "Error",
            "last_education": "Error",
            "total_experience": "Error",
            "present_field": "Error",
            "overall_expertise_area": "Error",
            "present_organization_designation": "Error",
            "research_experience": "Error",
            "achievements": "Error",
            "present_organization_name": "Error",
            "working_experience_in_present_organization": "Error",
            "working_experience_in_year_in_present_organization": "Error",
            "mobile": "Error",
            "email": "Error"
        }

# Function to find all PDF files in the directory structure
def find_pdf_files(root_path):
    pdf_files = []
    for root, _, files in os.walk(root_path):
        for file in files:
            if file.lower().endswith('.pdf'):
                # Get relative path components
                rel_path = os.path.relpath(root, root_path)
                if rel_path == '.':
                    subfolder = ""
                else:
                    subfolder = rel_path

                pdf_files.append({
                    'full_path': os.path.join(root, file),
                    'filename': file,
                    'root_folder': root_path,
                    'subfolder': subfolder
                })
    return pdf_files

# Main execution
def main():
    print(f"Starting CV analysis from root path: {root_path}")

    # Find all PDF files
    pdf_files = find_pdf_files(root_path)
    print(f"Found {len(pdf_files)} PDF files to process")

    # Create a list to store results
    all_results = []

    # Process each PDF file
    for pdf_info in tqdm(pdf_files, desc="Processing CVs"):
        try:
            # Open the PDF file and extract text
            with open(pdf_info['full_path'], 'rb') as file:
                # Extract text from PDF
                cv_text = extract_text_from_pdf(file)

                # Extract structured information
                cv_info = extract_cv_info(cv_text)

                # Add file information
                cv_info["filename"] = pdf_info['filename']
                cv_info["root_folder"] = pdf_info['root_folder']
                cv_info["subfolder"] = pdf_info['subfolder']

                # Add to results list
                all_results.append(cv_info)

                # Print progress
                print(f"Processed: {pdf_info['filename']}")
        except Exception as e:
            print(f"Error processing {pdf_info['filename']}: {str(e)}")
            # Add error entry
            error_info = {
                "name": "Error",
                "last_education": "Error",
                "total_experience": "Error",
                "present_field": "Error",
                "overall_expertise_area": "Error",
                "present_organization_designation": "Error",
                "research_experience": "Error",
                "achievements": "Error",
                "present_organization_name": "Error",
                "working_experience_in_present_organization": "Error",
                "working_experience_in_year_in_present_organization": "Error",
                "mobile": "Error",
                "email": "Error",
                "filename": pdf_info['filename'],
                "root_folder": pdf_info['root_folder'],
                "subfolder": pdf_info['subfolder']
            }
            all_results.append(error_info)

    # Create a DataFrame from all results
    df = pd.DataFrame(all_results)

    # Define the desired column order
    column_order = [
        "name",
        "last_education",
        "overall_expertise_area",
        "present_organization_name",
        "present_organization_designation",
        "working_experience_in_present_organization",
        "working_experience_in_year_in_present_organization",
        "total_experience",
        "present_field",
        "research_experience",
        "achievements",
        "mobile",
        "email",
        "filename",
        "root_folder",
        "subfolder"
    ]

    # Reorder columns (only include columns that exist)
    existing_columns = [col for col in column_order if col in df.columns]
    extra_columns = [col for col in df.columns if col not in column_order]
    df = df[existing_columns + extra_columns]

    # Ensure all data is treated as strings to avoid conversion issues
    for column in df.columns:
        df[column] = df[column].astype(str)

    # Save the DataFrame to Excel
    output_path = os.path.join(root_path, "cv_analysis_results.xlsx")
    df.to_excel(output_path, index=False)

    print(f"Analysis complete! Results saved to: {output_path}")
    return df

# Run the main function
if __name__ == "__main__":
    result_df = main()
    display(result_df)  # Display the results in the notebook

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Starting CV analysis from root path: /content/drive/MyDrive/JBC HR AI Assistant 
Found 64 PDF files to process


Processing CVs:   0%|          | 0/64 [00:00<?, ?it/s]

Processed: Md__Golam_Mostofa_s_CV.pdf
Processed: Md_Bikasuzzaman_Resume.pdf
Processed: sazzad_s resume.pdf
Processed: CV - Utshab Kumar Ghosh - ML Engineer & Researcher.pdf
Processed: JABED CV.pdf
Processed: MD_ATIQUR_RAHMAN.pdf
Processed: Resume (7).pdf
Processed: CV_Niaz_Mohaiman_Abtahi.pdf
Processed: Sajid_Uzzaman.pdf
Processed: Saiyera_Rehmina.pdf
Processed: Arifuzzaman_Joy.pdf
Processed: Sourav_Saha.pdf
Processed: Asif_Iqbal.pdf
Processed: Ab_Kaium.pdf
Processed: Faisal_Ahmed.pdf
Processed: MdShariar_Hossain.pdf
Processed: Sowmik_Sarker.pdf
Processed: Mizanur_Rahman.pdf
Processed: Md (1).pdf
Processed: Fannana_Fahreen.pdf
Processed: Jawad_Mohmmad.pdf
Processed: SKZOBAIDUR_RAHMAN.pdf
Processed: Rifat_Sanaullah.pdf
Processed: MD (2).pdf
Processed: AlfajUddin_Ahmed.pdf
Processed: MohammadHossen_Sohrawardy.pdf
Processed: Tahsin_Masruf.pdf
Processed: FatemaTujJohora_Faria.pdf
Processed: ShanjidulIslam_Sadhin.pdf
Processed: Md (3).pdf
Processed: Shadman_Rohan.pdf
Processed: Ahsan_HabibT

,name,last_education,overall_expertise_area,present_organization_name,present_organization_designation,working_experience_in_present_organization,working_experience_in_year_in_present_organization,total_experience,present_field,research_experience,achievements,mobile,email,filename,root_folder,subfolder
0,Md. Golam Mostofa,"B.Sc in Computer Science and Engineering, Dhak...","Machine Learning, Deep Learning, Python Backen...",DevolvedAI,Machine Learning Engineer,September 2023,1 year 8 months,1.5 years,Machine Learning,M-DBSCAN: Modified DBSCAN Clustering Algorithm...,"Kaggle Notebooks Expert, Maximum Competition T...",+8801780739705,golammostofa10001@gmail.com,Md__Golam_Mostofa_s_CV.pdf,/content/drive/MyDrive/JBC HR AI Assistant,22.04
1,Md Bikasuzzaman,Bachelor of Engineering in Information and Com...,"AI, Machine Learning, Computer Vision, Deep Le...",Sysnova Information Systems Limited,AI Programmer (Computer Vision),February 2025,3 months,6 years,Computer Vision,"Research Assistant at ICE Innovation Lab, Isla...","Position: Top 2% (Team: Tensor Titans, Within ...",+8801997515363,bikasictiu1718@gmail.com,Md_Bikasuzzaman_Resume.pdf,/content/drive/MyDrive/JBC HR AI Assistant,22.04
2,MD SAZZAD HOSSAIN,"B.Sc. in Computer Science and Engineering, Daf...","Machine Learning, Computer Vision, Natural Lan...",Devolved AI,Machine Learning Engineer,March 2024,1 year 2 months,3,Machine Learning,Co-authored 'ML_SPS: Stroke Prediction System ...,Daffodil Inter-department Math Olympiad 2019 2...,+88019715-12153,sazzad1779@gmail.com,sazzad_s resume.pdf,/content/drive/MyDrive/JBC HR AI Assistant,22.04
3,Utshab Kumar Ghosh,B.Sc. in Computer Science & Engineering (CSE) ...,"Data Science, Machine Learning, Deep Learning,...",Nascenia Ltd.,Machine Learning Engineer,December 2022,2 years 5 months,4 years,Machine Learning,AI Research Assistant at MyMedicalHUB Int. (Oc...,Kaggle Expert (x3); Gold Medalist in National ...,(+880) 1701-067875,kumarutshab@gmail.com,CV - Utshab Kumar Ghosh - ML Engineer & Resear...,/content/drive/MyDrive/JBC HR AI Assistant,22.04
4,Md Jabed Hosen,Bachelor of Science in Computer Science and En...,"Problem Solving, OOP, Data Structures & Algori...",AtherZen IT Ltd,ML Engineer,July 2024,10 months,2,Machine Learning,Developed metalearning framework for medical i...,"Winner, RoboTech Olympiad 2023; Top 50 Partici...",+8801706276447,jabedhosen.cse@gmail.com,JABED CV.pdf,/content/drive/MyDrive/JBC HR AI Assistant,22.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,Al-Farabi Akash,"Master's degree, Applied Statistics and Data S...","Data Science, AI & LLM Development, Python Dev...",Cloud Tech One,LLM and Full-stack Developer,April 2024,1 year 1 month,4 years 6 months,LLM and Full-stack Development,"Graduate thesis in Machine Learning, Deep Lear...",Not found,+8801703212426,alfa.farabi@gmail.com,Al-Farabi_Akash.pdf,/content/drive/MyDrive/JBC HR AI Assistant,23.04
60,Saidur Rahman,Bachelor of Science in Electrical and Electron...,"Robotics, Automation, Digital Signal Processin...",Bangladesh University of Engineering and Techn...,BSc Student,December 2023,1 year 5 months,1 year 5 months,Electrical and Electronics Engineering,Research Affiliate at the Collaborative Roboti...,"Dean’s List Scholarships, Board Scholarships",+8801312601063,saidurrahamanabir@gmail.com,Saidur_Rahman.pdf,/content/drive/MyDrive/JBC HR AI Assistant,23.04
61,Mushfiqul Islam Chowdhury,"Bachelor's degree, Computer Science and Engine...","Python, Artificial Intelligence, Machine Learn...","Pipeline, Inc.",Software Engineer,May 2022,3 years,5 years,Software Engineering,Not found,"Runner up, Intra-University Programming Contes...",+8801515248558,nnahid878@gmail.com,MushfiqulIslam_Chowdhury.pdf,/content/drive/MyDrive/JBC HR AI Assistant,23.04
62,Md Arian Rahman,"Bachelor's degree, Computer Science, North Sou...","ML & AI, Tech Stack, DevOps & MLOps",Beraten Software,Software Engineer,July 2023,1 year 10 months,2 years 4 months,AI Develop

In [ ]:
# Install required packages
!pip install pandas
!pip install PyPDF2
!pip install openai
!pip install python-dateutil
!pip install tqdm
!pip install google-api-python-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.7 MB/s eta 0:00:00


In [ ]:


# CV Analyzer for Google Drive - JBC HR AI Assistant
# Mount Google Drive, extract CV information from PDFs, and create an Excel output file

import os
import pandas as pd
import io
import PyPDF2
import openai
import re
import tempfile
import json
from datetime import datetime
import dateutil.parser
from dateutil.relativedelta import relativedelta
from google.colab import drive
from tqdm.notebook import tqdm
from googleapiclient.discovery import build
from google.colab import auth
from google.auth import default

# Mount Google Drive
drive.mount('/content/drive')

# Set your OpenAI API key
openai_api_key = ""
openai.api_key = openai_api_key

# Define the root path
root_path = "/content/drive/MyDrive/JBC_HR_AI_ASSISTANT"

# Google Drive file link base URL
DRIVE_LINK_BASE = "https://drive.google.com/file/d/"

# Function to get Google Drive file ID
def get_file_id(file_path):
    """Get Google Drive file ID from local path"""
    try:
        # Authenticate and create the Drive API client
        auth.authenticate_user()
        creds, _ = default()
        drive_service = build('drive', 'v3', credentials=creds)

        # Extract the relative path from the full path
        relative_path = os.path.relpath(file_path, "/content/drive/MyDrive")

        # Search for the file by name
        filename = os.path.basename(file_path)
        query = f"name = '{filename}' and trashed = false"

        # Execute the query
        results = drive_service.files().list(
            q=query,
            spaces='drive',
            fields='files(id, name, parents)'
        ).execute()

        items = results.get('files', [])

        if not items:
            return None

        # If multiple files have the same name, try to match the path
        if len(items) > 1:
            for item in items:
                # Get the file's complete path
                file_path_in_drive = get_file_path_in_drive(drive_service, item['id'])
                if relative_path in file_path_in_drive:
                    return item['id']

            # If no path match, return the first one
            return items[0]['id']
        else:
            return items[0]['id']
    except Exception as e:
        print(f"Error getting file ID: {str(e)}")
        return None

# Function to get file path in Drive
def get_file_path_in_drive(service, file_id):
    """Get the file path in Google Drive"""
    try:
        # Get the file metadata
        file = service.files().get(fileId=file_id, fields='name, parents').execute()

        path = [file['name']]

        # Get all parent folders
        if 'parents' in file:
            parent_id = file['parents'][0]
            while parent_id:
                parent = service.files().get(fileId=parent_id, fields='name, parents').execute()
                path.insert(0, parent['name'])

                if 'parents' in parent:
                    parent_id = parent['parents'][0]
                else:
                    parent_id = None

        return '/'.join(path)
    except Exception as e:
        print(f"Error getting file path: {str(e)}")
        return ""

# Function to create shareable link
def create_shareable_link(file_id):
    """Create a shareable link for the Google Drive file"""
    if file_id:
        return f"{DRIVE_LINK_BASE}{file_id}/view?usp=sharing"
    return "Link not available"

def extract_text_from_pdf(pdf_file):
    """Extract text content from a PDF file."""
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    text = ""
    for page_num in range(len(pdf_reader.pages)):
        text += pdf_reader.pages[page_num].extract_text()
    return text

def calculate_experience_duration(start_date_str):
    """Calculate duration between start date and current date in 'X year Y month' format."""
    try:
        # Parse the start date string
        if start_date_str == "Not found" or not start_date_str:
            return "Not found"

        # Try to parse the date with dateutil parser
        try:
            start_date = dateutil.parser.parse(start_date_str, fuzzy=True)
        except:
            # If parsing fails, try to extract month and year manually
            match = re.search(r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* (\d{4})',
                             start_date_str, re.IGNORECASE)
            if match:
                month_str = match.group(1)
                year_str = match.group(2)
                # Map abbreviated month to number
                month_map = {
                    'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4,
                    'may': 5, 'jun': 6, 'jul': 7, 'aug': 8,
                    'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
                }
                month = month_map.get(month_str.lower()[:3], 1)
                year = int(year_str)
                start_date = datetime(year, month, 1)
            else:
                return "Date format not recognized"

        # Calculate the difference between the start date and current date
        current_date = datetime.now()
        delta = relativedelta(current_date, start_date)

        # Format the result as "X year Y month"
        years = delta.years
        months = delta.months

        if years == 0:
            if months == 1:
                return f"{months} month"
            else:
                return f"{months} months"
        elif years == 1:
            if months == 0:
                return "1 year"
            elif months == 1:
                return "1 year 1 month"
            else:
                return f"1 year {months} months"
        else:
            if months == 0:
                return f"{years} years"
            elif months == 1:
                return f"{years} years 1 month"
            else:
                return f"{years} years {months} months"
    except Exception as e:
        return f"Error calculating duration: {str(e)}"

def extract_field(text, field_name):
    """Extract a specific field from text response when JSON parsing fails"""
    pattern = rf"{field_name}[:\s]+(.*?)(?:\n|$|,)"
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
        return match.group(1).strip()
    return "Not found"

def extract_cv_info(cv_text):
    """Use OpenAI API to extract structured information from CV text."""

    # Get current date for calculating work experience
    current_date = datetime.now()
    current_date_str = current_date.strftime("%Y %B")

    prompt = f"""
    Extract the following information from the CV text below.
    If you cannot find a particular piece of information, respond with "Not found" for that field.

    Information to extract:
    1. Name
    2. Last Education and university
    3. Number of total year experiences
    4. Present field of experience
    5. Overall expertise area
    6. Present organization designation
    7. Research experience (any research positions, publications, or projects)
    8. Achievements (awards, recognitions, significant accomplishments)
    9. Mobile number
    10. Email address
    11. Present organization name
    12. Working experience in present organization (start date in format 'Month YYYY', e.g. 'December 2022')

    Today is {current_date_str}.

    CV Text:
    {cv_text}

    Your response MUST be a valid JSON object with ONLY the following keys:
    {{
      "name": "extracted name",
      "last_education": "extracted education and university",
      "total_experience": "total number of experiences in all organizations",
      "present_field": "present field of experience",
      "overall_expertise_area": "areas of expertise or specialization",
      "present_organization_designation": "current job title or designation",
      "research_experience": "details of research experience if any",
      "achievements": "major achievements and awards if any",
      "mobile": "extracted mobile number",
      "email": "extracted email address",
      "present_organization_name": "name of current organization",
      "working_experience_in_present_organization": "start date in format 'Month YYYY'"
    }}

    Do not include any explanation, just return the JSON object.
    """

    try:
        # Use GPT-4o model for better extraction
        model = "gpt-4o"

        response = openai.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant that extracts structured information from CVs. Return only valid JSON."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3  # Lower temperature for more consistent results
        )

        # Extract and parse the JSON response
        result = response.choices[0].message.content

        # Try to find JSON in the response
        json_match = re.search(r'(\{[\s\S]*\})', result, re.DOTALL)
        if json_match:
            json_str = json_match.group(1)
            try:
                parsed_result = json.loads(json_str)

                # Calculate work experience duration
                start_date = parsed_result.get("working_experience_in_present_organization", "Not found")
                parsed_result["working_experience_in_year_in_present_organization"] = calculate_experience_duration(start_date)

                return parsed_result
            except json.JSONDecodeError:
                print(f"Failed to parse JSON from response. Attempting alternate extraction.")

        # If extraction failed, try to create a structured response manually
        try:
            # Create a standard response manually
            start_date = extract_field(result, "working_experience_in_present_organization")
            experience_duration = calculate_experience_duration(start_date)

            # Return in the specified order
            return {
                "name": extract_field(result, "name"),
                "last_education": extract_field(result, "last_education"),
                "overall_expertise_area": extract_field(result, "overall_expertise_area"),
                "present_organization_name": extract_field(result, "present_organization_name"),
                "present_organization_designation": extract_field(result, "present_organization_designation"),
                "working_experience_in_present_organization": start_date,
                "working_experience_in_year_in_present_organization": experience_duration,
                "total_experience": extract_field(result, "total_experience"),
                "present_field": extract_field(result, "present_field"),
                "research_experience": extract_field(result, "research_experience"),
                "achievements": extract_field(result, "achievements"),
                "mobile": extract_field(result, "mobile"),
                "email": extract_field(result, "email")
            }
        except Exception as e:
            print(f"Error creating structured response: {str(e)}")
            # Last resort, try direct JSON parsing
            parsed_result = json.loads(result)

            # Calculate work experience duration
            start_date = parsed_result.get("working_experience_in_present_organization", "Not found")
            parsed_result["working_experience_in_year_in_present_organization"] = calculate_experience_duration(start_date)

            return parsed_result

    except Exception as e:
        print(f"Error extracting information: {str(e)}")
        return {
            "name": "Error",
            "last_education": "Error",
            "total_experience": "Error",
            "present_field": "Error",
            "overall_expertise_area": "Error",
            "present_organization_designation": "Error",
            "research_experience": "Error",
            "achievements": "Error",
            "present_organization_name": "Error",
            "working_experience_in_present_organization": "Error",
            "working_experience_in_year_in_present_organization": "Error",
            "mobile": "Error",
            "email": "Error"
        }

# Function to find all PDF files in the directory structure
def find_pdf_files(root_path):
    pdf_files = []
    for root, _, files in os.walk(root_path):
        for file in files:
            if file.lower().endswith('.pdf'):
                # Get relative path components
                rel_path = os.path.relpath(root, root_path)
                if rel_path == '.':
                    subfolder = ""
                else:
                    subfolder = rel_path

                full_path = os.path.join(root, file)

                # Get file ID and create shareable link
                file_id = get_file_id(full_path)
                cv_link = create_shareable_link(file_id)

                pdf_files.append({
                    'full_path': full_path,
                    'filename': file,
                    'root_folder': root_path,
                    'subfolder': subfolder,
                    'cv_link': cv_link
                })
    return pdf_files

# Main execution
def main():
    print(f"Starting CV analysis from root path: {root_path}")

    # Find all PDF files
    pdf_files = find_pdf_files(root_path)
    print(f"Found {len(pdf_files)} PDF files to process")

    # Create a list to store results
    all_results = []

    # Process each PDF file
    for pdf_info in tqdm(pdf_files, desc="Processing CVs"):
        try:
            # Open the PDF file and extract text
            with open(pdf_info['full_path'], 'rb') as file:
                # Extract text from PDF
                cv_text = extract_text_from_pdf(file)

                # Extract structured information
                cv_info = extract_cv_info(cv_text)

                # Add file information
                cv_info["filename"] = pdf_info['filename']
                cv_info["root_folder"] = pdf_info['root_folder']
                cv_info["subfolder"] = pdf_info['subfolder']
                cv_info["cv_link"] = pdf_info['cv_link']

                # Add to results list
                all_results.append(cv_info)

                # Print progress
                print(f"Processed: {pdf_info['filename']}")
        except Exception as e:
            print(f"Error processing {pdf_info['filename']}: {str(e)}")
            # Add error entry
            error_info = {
                "name": "Error",
                "last_education": "Error",
                "total_experience": "Error",
                "present_field": "Error",
                "overall_expertise_area": "Error",
                "present_organization_designation": "Error",
                "research_experience": "Error",
                "achievements": "Error",
                "present_organization_name": "Error",
                "working_experience_in_present_organization": "Error",
                "working_experience_in_year_in_present_organization": "Error",
                "mobile": "Error",
                "email": "Error",
                "filename": pdf_info['filename'],
                "root_folder": pdf_info['root_folder'],
                "subfolder": pdf_info['subfolder'],
                "cv_link": pdf_info['cv_link']
            }
            all_results.append(error_info)

    # Create a DataFrame from all results
    df = pd.DataFrame(all_results)

    # Define the desired column order
    column_order = [
        "name",
        "last_education",
        "overall_expertise_area",
        "present_organization_name",
        "present_organization_designation",
        "working_experience_in_present_organization",
        "working_experience_in_year_in_present_organization",
        "total_experience",
        "present_field",
        "research_experience",
        "achievements",
        "mobile",
        "email",
        "filename",
        "root_folder",
        "subfolder",
        "cv_link"
    ]

    # Reorder columns (only include columns that exist)
    existing_columns = [col for col in column_order if col in df.columns]
    extra_columns = [col for col in df.columns if col not in column_order]
    df = df[existing_columns + extra_columns]

    # Ensure all data is treated as strings to avoid conversion issues
    for column in df.columns:
        df[column] = df[column].astype(str)

    # Save the DataFrame to Excel
    output_path = os.path.join(root_path, "cv_analysis_results.xlsx")
    df.to_excel(output_path, index=False)

    print(f"Analysis complete! Results saved to: {output_path}")
    return df

# Run the main function
if __name__ == "__main__":
    result_df = main()
    display(result_df)  # Display the results in the notebook

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Starting CV analysis from root path: /content/drive/MyDrive/JBC_HR_AI_ASSISTANT
Found 5 PDF files to process


Processing CVs:   0%|          | 0/5 [00:00<?, ?it/s]

Processed: Md Al Amin Tokder.pdf
Processed: resume-kaiser.pdf
Processed: resume_rizwan.pdf
Processed: Abu Ahamed Rafi-1.pdf
Processed: Asif_Iqbal.pdf
Analysis complete! Results saved to: /content/drive/MyDrive/JBC_HR_AI_ASSISTANT/cv_analysis_results.xlsx


,name,last_education,overall_expertise_area,present_organization_name,present_organization_designation,working_experience_in_present_organization,working_experience_in_year_in_present_organization,total_experience,present_field,research_experience,achievements,mobile,email,filename,root_folder,subfolder,cv_link
0,MD AL AMIN TOKDER,"BSc in Computer Science and Engineering, Rajsh...","Machine Learning, Artificial Intelligence, Gen...",JB Connect Ltd,Machine Learning Engineer,January 2024,1 year 4 months,4 years,Machine Learning,Publications: Garbage Classification using a T...,"ICPC Dhaka Site-2023, Innovative Idea and Proj...",+8801750206042,alamintokdercse@gmail.com,Md Al Amin Tokder.pdf,/content/drive/MyDrive/JBC_HR_AI_ASSISTANT,22.04,https://drive.google.com/file/d/12KIfy3xxAjibN...
1,Kaiser Hamid Rabbi,"Master of Science, Otto-von-Guericke Universit...","Computer Vision, Natural Language Processing, ...",TigerIT,Senior Software Engineer,October 2019,5 years 7 months,6 years,Machine Learning and Artificial Intelligence,Author of 4 AI & Deep Learning courses on Udem...,Boosted number plate detection & tracking accu...,+8801644611605,kaiser.hamid.rabbi@gmail.com,resume-kaiser.pdf,/content/drive/MyDrive/JBC_HR_AI_ASSISTANT,22.04,https://drive.google.com/file/d/1jhuHCkAArjfia...
2,Syed Rizwan,"B.Sc in Computer Science & Engineering, Jashor...","Backend development, system design, Ruby on Ra...",JB Connect,Backend Software Engineer,January 2025,4 months,2 years,Backend Software Engineering,"Publications: Syed Rizwan, Md. Shazzad Ali Sob...",Finalist in IT based Business Case Competition...,01740404338,perizwansm@outlook.com,resume_rizwan.pdf,/content/drive/MyDrive/JBC_HR_AI_ASSISTANT,22.04,https://drive.google.com/file/d/1b7NGiz2QqKgVX...
3,Abu Ahamed Rafi,"B.Sc. in Computer Science and Engineering, Ind...","Web development, database management, API inte...",Computer Services Ltd.,Trainee Software Engineer,February 2024,1 year 3 months,1 year and 3 months,Software Engineering,Not found,Not found,+8801787842209,abuahamedrafi@gmail.com,Abu Ahamed Rafi-1.pdf,/content/drive/MyDrive/JBC_HR_AI_ASSISTANT,23.04,https://drive.google.com/file/d/1E49NmKn0TUdit...
4,Asif Iqbal,"Masters, Business Administration and Managemen...","blockchain, AI, full-stack web development",Meta Together,Full Stack Developer,January 2021,4 years 4 months,5 years 11 months,Full Stack Development,Not found,Not found,+8801675259212,asifdotpy@gmail.com,Asif_Iqbal.pdf,/content/drive/MyDrive/JBC_HR_AI_ASSISTANT,23.04,https://drive.google.com/file/d/1Y8gZGTpMhbFqr...


In [ ]:
# Install required packages
!pip install pandas
!pip install PyPDF2
!pip install openai
!pip install python-dateutil
!pip install tqdm
!pip install google-api-python-client
!pip install openpyxl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.6 MB/s eta 0:00:00


In [ ]:

# CV Analyzer for Google Drive - JBC HR AI Assistant
# Mount Google Drive, extract CV information from PDFs, and create an Excel output file

import os
import pandas as pd
import io
import PyPDF2
import openai
import re
import tempfile
import json
from datetime import datetime
import dateutil.parser
from dateutil.relativedelta import relativedelta
from google.colab import drive
from tqdm.notebook import tqdm
from googleapiclient.discovery import build
from google.colab import auth
from google.auth import default
import openpyxl
from openpyxl.styles import Font
from openpyxl.styles.colors import Color

# Mount Google Drive
drive.mount('/content/drive')

# Set your OpenAI API key
openai_api_key = ""
openai.api_key = openai_api_key

# Define the root path
root_path = "/content/drive/MyDrive/JBC_HR_AI_ASSISTANT/22.04"

# Google Drive file link base URL
DRIVE_LINK_BASE = "https://drive.google.com/file/d/"

# Function to get Google Drive file ID
def get_file_id(file_path):
    """Get Google Drive file ID from local path"""
    try:
        # Authenticate and create the Drive API client
        auth.authenticate_user()
        creds, _ = default()
        drive_service = build('drive', 'v3', credentials=creds)

        # Extract the relative path from the full path
        relative_path = os.path.relpath(file_path, "/content/drive/MyDrive")

        # Search for the file by name
        filename = os.path.basename(file_path)
        query = f"name = '{filename}' and trashed = false"

        # Execute the query
        results = drive_service.files().list(
            q=query,
            spaces='drive',
            fields='files(id, name, parents)'
        ).execute()

        items = results.get('files', [])

        if not items:
            return None

        # If multiple files have the same name, try to match the path
        if len(items) > 1:
            for item in items:
                # Get the file's complete path
                file_path_in_drive = get_file_path_in_drive(drive_service, item['id'])
                if relative_path in file_path_in_drive:
                    return item['id']

            # If no path match, return the first one
            return items[0]['id']
        else:
            return items[0]['id']
    except Exception as e:
        print(f"Error getting file ID: {str(e)}")
        return None

# Function to get file path in Drive
def get_file_path_in_drive(service, file_id):
    """Get the file path in Google Drive"""
    try:
        # Get the file metadata
        file = service.files().get(fileId=file_id, fields='name, parents').execute()

        path = [file['name']]

        # Get all parent folders
        if 'parents' in file:
            parent_id = file['parents'][0]
            while parent_id:
                parent = service.files().get(fileId=parent_id, fields='name, parents').execute()
                path.insert(0, parent['name'])

                if 'parents' in parent:
                    parent_id = parent['parents'][0]
                else:
                    parent_id = None

        return '/'.join(path)
    except Exception as e:
        print(f"Error getting file path: {str(e)}")
        return ""

# Function to create shareable link
def create_shareable_link(file_id):
    """Create a shareable link for the Google Drive file"""
    if file_id:
        return f"{DRIVE_LINK_BASE}{file_id}/view?usp=sharing"
    return "Link not available"

def extract_text_from_pdf(pdf_file):
    """Extract text content from a PDF file."""
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    text = ""
    for page_num in range(len(pdf_reader.pages)):
        text += pdf_reader.pages[page_num].extract_text()
    return text

def calculate_experience_duration(start_date_str):
    """Calculate duration between start date and current date in 'X year Y month' format."""
    try:
        # Parse the start date string
        if start_date_str == "Not found" or not start_date_str:
            return "Not found"

        # Try to parse the date with dateutil parser
        try:
            start_date = dateutil.parser.parse(start_date_str, fuzzy=True)
        except:
            # If parsing fails, try to extract month and year manually
            match = re.search(r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* (\d{4})',
                             start_date_str, re.IGNORECASE)
            if match:
                month_str = match.group(1)
                year_str = match.group(2)
                # Map abbreviated month to number
                month_map = {
                    'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4,
                    'may': 5, 'jun': 6, 'jul': 7, 'aug': 8,
                    'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
                }
                month = month_map.get(month_str.lower()[:3], 1)
                year = int(year_str)
                start_date = datetime(year, month, 1)
            else:
                return "Date format not recognized"

        # Calculate the difference between the start date and current date
        current_date = datetime.now()
        delta = relativedelta(current_date, start_date)

        # Format the result as "X year Y month"
        years = delta.years
        months = delta.months

        if years == 0:
            if months == 1:
                return f"{months} month"
            else:
                return f"{months} months"
        elif years == 1:
            if months == 0:
                return "1 year"
            elif months == 1:
                return "1 year 1 month"
            else:
                return f"1 year {months} months"
        else:
            if months == 0:
                return f"{years} years"
            elif months == 1:
                return f"{years} years 1 month"
            else:
                return f"{years} years {months} months"
    except Exception as e:
        return f"Error calculating duration: {str(e)}"

def extract_field(text, field_name):
    """Extract a specific field from text response when JSON parsing fails"""
    pattern = rf"{field_name}[:\s]+(.*?)(?:\n|$|,)"
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
        return match.group(1).strip()
    return "Not found"

def extract_cv_info(cv_text):
    """Use OpenAI API to extract structured information from CV text."""

    # Get current date for calculating work experience
    current_date = datetime.now()
    current_date_str = current_date.strftime("%Y %B")

    prompt = f"""
    Extract the following information from the CV text below.
    If you cannot find a particular piece of information, respond with "Not found" for that field.

    Information to extract:
    1. Name
    2. Last Education and university
    3. Number of total year experiences
    4. Present field of experience
    5. Overall expertise area
    6. Present organization designation
    7. Research experience (any research positions, publications, or projects)
    8. Achievements (awards, recognitions, significant accomplishments)
    9. Mobile number
    10. Email address
    11. Present organization name
    12. Working experience in present organization (start date in format 'Month YYYY', e.g. 'December 2022')

    Today is {current_date_str}.

    CV Text:
    {cv_text}

    Your response MUST be a valid JSON object with ONLY the following keys:
    {{
      "name": "extracted name",
      "last_education": "extracted education and university",
      "total_experience": "total number of experiences in all organizations",
      "present_field": "present field of experience",
      "overall_expertise_area": "areas of expertise or specialization",
      "present_organization_designation": "current job title or designation",
      "research_experience": "details of research experience if any",
      "achievements": "major achievements and awards if any",
      "mobile": "extracted mobile number",
      "email": "extracted email address",
      "present_organization_name": "name of current organization",
      "working_experience_in_present_organization": "start date in format 'Month YYYY'"
    }}

    Do not include any explanation, just return the JSON object.
    """

    try:
        # Use GPT-4o model for better extraction
        model = "gpt-4o"

        response = openai.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant that extracts structured information from CVs. Return only valid JSON."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3  # Lower temperature for more consistent results
        )

        # Extract and parse the JSON response
        result = response.choices[0].message.content

        # Try to find JSON in the response
        json_match = re.search(r'(\{[\s\S]*\})', result, re.DOTALL)
        if json_match:
            json_str = json_match.group(1)
            try:
                parsed_result = json.loads(json_str)

                # Calculate work experience duration
                start_date = parsed_result.get("working_experience_in_present_organization", "Not found")
                parsed_result["working_experience_in_year_in_present_organization"] = calculate_experience_duration(start_date)

                return parsed_result
            except json.JSONDecodeError:
                print(f"Failed to parse JSON from response. Attempting alternate extraction.")

        # If extraction failed, try to create a structured response manually
        try:
            # Create a standard response manually
            start_date = extract_field(result, "working_experience_in_present_organization")
            experience_duration = calculate_experience_duration(start_date)

            # Return in the specified order
            return {
                "name": extract_field(result, "name"),
                "last_education": extract_field(result, "last_education"),
                "overall_expertise_area": extract_field(result, "overall_expertise_area"),
                "present_organization_name": extract_field(result, "present_organization_name"),
                "present_organization_designation": extract_field(result, "present_organization_designation"),
                "working_experience_in_present_organization": start_date,
                "working_experience_in_year_in_present_organization": experience_duration,
                "total_experience": extract_field(result, "total_experience"),
                "present_field": extract_field(result, "present_field"),
                "research_experience": extract_field(result, "research_experience"),
                "achievements": extract_field(result, "achievements"),
                "mobile": extract_field(result, "mobile"),
                "email": extract_field(result, "email")
            }
        except Exception as e:
            print(f"Error creating structured response: {str(e)}")
            # Last resort, try direct JSON parsing
            parsed_result = json.loads(result)

            # Calculate work experience duration
            start_date = parsed_result.get("working_experience_in_present_organization", "Not found")
            parsed_result["working_experience_in_year_in_present_organization"] = calculate_experience_duration(start_date)

            return parsed_result

    except Exception as e:
        print(f"Error extracting information: {str(e)}")
        return {
            "name": "Error",
            "last_education": "Error",
            "total_experience": "Error",
            "present_field": "Error",
            "overall_expertise_area": "Error",
            "present_organization_designation": "Error",
            "research_experience": "Error",
            "achievements": "Error",
            "present_organization_name": "Error",
            "working_experience_in_present_organization": "Error",
            "working_experience_in_year_in_present_organization": "Error",
            "mobile": "Error",
            "email": "Error"
        }

# Function to find all PDF files in the directory structure
def find_pdf_files(root_path):
    pdf_files = []
    for root, _, files in os.walk(root_path):
        for file in files:
            if file.lower().endswith('.pdf'):
                # Get relative path components
                rel_path = os.path.relpath(root, root_path)
                if rel_path == '.':
                    subfolder = ""
                else:
                    subfolder = rel_path

                full_path = os.path.join(root, file)

                # Get file ID and create shareable link
                file_id = get_file_id(full_path)
                cv_link = create_shareable_link(file_id)

                pdf_files.append({
                    'full_path': full_path,
                    'filename': file,
                    'root_folder': root_path,
                    'subfolder': subfolder,
                    'cv_link': cv_link
                })
    return pdf_files

# Main execution
def main():
    print(f"Starting CV analysis from root path: {root_path}")

    # Find all PDF files
    pdf_files = find_pdf_files(root_path)
    print(f"Found {len(pdf_files)} PDF files to process")

    # Create a list to store results
    all_results = []

    # Process each PDF file
    for pdf_info in tqdm(pdf_files, desc="Processing CVs"):
        try:
            # Open the PDF file and extract text
            with open(pdf_info['full_path'], 'rb') as file:
                # Extract text from PDF
                cv_text = extract_text_from_pdf(file)

                # Extract structured information
                cv_info = extract_cv_info(cv_text)

                # Add file information
                cv_info["filename"] = pdf_info['filename']
                cv_info["root_folder"] = pdf_info['root_folder']
                cv_info["subfolder"] = pdf_info['subfolder']
                cv_info["cv_link"] = pdf_info['cv_link']

                # Add to results list
                all_results.append(cv_info)

                # Print progress
                print(f"Processed: {pdf_info['filename']}")
        except Exception as e:
            print(f"Error processing {pdf_info['filename']}: {str(e)}")
            # Add error entry
            error_info = {
                "name": "Error",
                "last_education": "Error",
                "total_experience": "Error",
                "present_field": "Error",
                "overall_expertise_area": "Error",
                "present_organization_designation": "Error",
                "research_experience": "Error",
                "achievements": "Error",
                "present_organization_name": "Error",
                "working_experience_in_present_organization": "Error",
                "working_experience_in_year_in_present_organization": "Error",
                "mobile": "Error",
                "email": "Error",
                "filename": pdf_info['filename'],
                "root_folder": pdf_info['root_folder'],
                "subfolder": pdf_info['subfolder'],
                "cv_link": pdf_info['cv_link']
            }
            all_results.append(error_info)

    # Create a DataFrame from all results
    df = pd.DataFrame(all_results)

    # Define the desired column order
    column_order = [
        "name",
        "last_education",
        "overall_expertise_area",
        "present_organization_name",
        "present_organization_designation",
        "working_experience_in_present_organization",
        "working_experience_in_year_in_present_organization",
        "total_experience",
        "present_field",
        "research_experience",
        "achievements",
        "mobile",
        "email",
        "filename",  # Filename will be the hyperlink
        "root_folder",
        "subfolder",
        "cv_link"
    ]

    # Reorder columns (only include columns that exist)
    existing_columns = [col for col in column_order if col in df.columns]
    extra_columns = [col for col in df.columns if col not in column_order]
    df = df[existing_columns + extra_columns]

    # Ensure all data is treated as strings to avoid conversion issues
    for column in df.columns:
        df[column] = df[column].astype(str)

    # Save the DataFrame to Excel
    output_path = os.path.join(root_path, "cv_analysis_results.xlsx")

    # First save using pandas to get the basic structure
    df.to_excel(output_path, index=False)

    # Now, modify the Excel file to create proper hyperlinks
    workbook = openpyxl.load_workbook(output_path)
    worksheet = workbook.active

    # Find the column indices for filename and cv_link
    header_row = worksheet[1]
    filename_col_idx = None
    cv_link_col_idx = None

    for idx, cell in enumerate(header_row, 1):
        if cell.value == "filename":
            filename_col_idx = idx
        elif cell.value == "cv_link":
            cv_link_col_idx = idx

    # If both columns exist, create hyperlinks
    if filename_col_idx and cv_link_col_idx:
        for row_idx in range(2, worksheet.max_row + 1):
            filename_cell = worksheet.cell(row=row_idx, column=filename_col_idx)
            link_cell = worksheet.cell(row=row_idx, column=cv_link_col_idx)

            if filename_cell.value and link_cell.value and link_cell.value != "Link not available":
                # Create a hyperlink for the filename that points to the cv_link URL
                filename = filename_cell.value
                link_url = link_cell.value

                # Set the hyperlink
                filename_cell.hyperlink = link_url
                filename_cell.value = filename

                # Format the cell as a hyperlink (blue and underlined)
                filename_cell.font = Font(color="0000FF", underline="single")

                # Clear the original link cell (optional - you could keep it if you want)
                link_cell.value = "Click filename to open CV"

    # Save the modified workbook
    workbook.save(output_path)

    print(f"Analysis complete! Results saved to: {output_path} with clickable hyperlinks")
    return df

# Run the main function
if __name__ == "__main__":
    result_df = main()
    # display(result_df)  # Display the results in the notebook

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Starting CV analysis from root path: /content/drive/MyDrive/JBC_HR_AI_ASSISTANT/22.04
Found 3 PDF files to process


Processing CVs:   0%|          | 0/3 [00:00<?, ?it/s]

Processed: Md Al Amin Tokder.pdf
Processed: resume-kaiser.pdf
Processed: resume_rizwan.pdf
Analysis complete! Results saved to: /content/drive/MyDrive/JBC_HR_AI_ASSISTANT/22.04/cv_analysis_results.xlsx with clickable hyperlinks


,name,last_education,overall_expertise_area,present_organization_name,present_organization_designation,working_experience_in_present_organization,working_experience_in_year_in_present_organization,total_experience,present_field,research_experience,achievements,mobile,email,filename,root_folder,subfolder,cv_link
0,MD AL AMIN TOKDER,"BSc in Computer Science and Engineering, Rajsh...","Machine Learning, Artificial Intelligence, Web...",JB Connect Ltd,Machine Learning Engineer,January 2024,1 year 4 months,4,Machine Learning,Publications: Garbage Classification using a T...,"ICPC Dhaka Site-2023, Innovative Idea and Proj...",+8801750206042,alamintokdercse@gmail.com,Md Al Amin Tokder.pdf,/content/drive/MyDrive/JBC_HR_AI_ASSISTANT/22.04,,https://drive.google.com/file/d/12KIfy3xxAjibN...
1,Kaiser Hamid Rabbi,"Master of Science, Otto-von-Guericke Universit...","Computer Vision, Natural Language Processing, ...",TigerIT,Senior Software Engineer,October 2019,5 years 7 months,6 years,Artificial Intelligence and Machine Learning,Author of 4 AI & Deep Learning courses on Udem...,Boosted number plate detection & tracking accu...,+8801644611605,kaiser.hamid.rabbi@gmail.com,resume-kaiser.pdf,/content/drive/MyDrive/JBC_HR_AI_ASSISTANT/22.04,,https://drive.google.com/file/d/1jhuHCkAArjfia...
2,Syed Rizwan,"B.Sc in Computer Science & Engineering, Jashor...","Backend development, system design, Ruby on Ra...",JB Connect,Backend Software Engineer,January 2025,4 months,3 years,Backend Software Engineering,"Publications: 1. Syed Rizwan, Md. Shazzad Ali ...",Finalist in IT based Business Case Competition...,01740404338,perizwansm@outlook.com,resume_rizwan.pdf,/content/drive/MyDrive/JBC_HR_AI_ASSISTANT/22.04,,https://drive.google.com/file/d/1b7NGiz2QqKgVX...


In [ ]:


# CV Analyzer for Google Drive - JBC HR AI Assistant
# Mount Google Drive, extract CV information from PDFs, and create an Excel output file

import os
import pandas as pd
import io
import PyPDF2
import openai
import re
import tempfile
import json
from datetime import datetime
import dateutil.parser
from dateutil.relativedelta import relativedelta
from google.colab import drive
from tqdm.notebook import tqdm
from googleapiclient.discovery import build
from google.colab import auth
from google.auth import default
import openpyxl
from openpyxl.styles import Font
from openpyxl.styles.colors import Color
from google.colab import files

# Mount Google Drive
drive.mount('/content/drive')

# Set your OpenAI API key
openai_api_key = ""
openai.api_key = openai_api_key

# Define the root path
root_path = "/content/drive/MyDrive/JBC_HR_AI_ASSISTANT/22.04"

# Google Drive file link base URL
DRIVE_LINK_BASE = "https://drive.google.com/file/d/"

# Function to get Google Drive file ID
def get_file_id(file_path):
    """Get Google Drive file ID from local path"""
    try:
        # Authenticate and create the Drive API client
        auth.authenticate_user()
        creds, _ = default()
        drive_service = build('drive', 'v3', credentials=creds)

        # Extract the relative path from the full path
        relative_path = os.path.relpath(file_path, "/content/drive/MyDrive")

        # Search for the file by name
        filename = os.path.basename(file_path)
        query = f"name = '{filename}' and trashed = false"

        # Execute the query
        results = drive_service.files().list(
            q=query,
            spaces='drive',
            fields='files(id, name, parents)'
        ).execute()

        items = results.get('files', [])

        if not items:
            return None

        # If multiple files have the same name, try to match the path
        if len(items) > 1:
            for item in items:
                # Get the file's complete path
                file_path_in_drive = get_file_path_in_drive(drive_service, item['id'])
                if relative_path in file_path_in_drive:
                    return item['id']

            # If no path match, return the first one
            return items[0]['id']
        else:
            return items[0]['id']
    except Exception as e:
        print(f"Error getting file ID: {str(e)}")
        return None

# Function to get file path in Drive
def get_file_path_in_drive(service, file_id):
    """Get the file path in Google Drive"""
    try:
        # Get the file metadata
        file = service.files().get(fileId=file_id, fields='name, parents').execute()

        path = [file['name']]

        # Get all parent folders
        if 'parents' in file:
            parent_id = file['parents'][0]
            while parent_id:
                parent = service.files().get(fileId=parent_id, fields='name, parents').execute()
                path.insert(0, parent['name'])

                if 'parents' in parent:
                    parent_id = parent['parents'][0]
                else:
                    parent_id = None

        return '/'.join(path)
    except Exception as e:
        print(f"Error getting file path: {str(e)}")
        return ""

# Function to create shareable link
def create_shareable_link(file_id):
    """Create a shareable link for the Google Drive file"""
    if file_id:
        return f"{DRIVE_LINK_BASE}{file_id}/view?usp=sharing"
    return "Link not available"

def extract_text_from_pdf(pdf_file):
    """Extract text content from a PDF file."""
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    text = ""
    for page_num in range(len(pdf_reader.pages)):
        text += pdf_reader.pages[page_num].extract_text()
    return text

def calculate_experience_duration(start_date_str):
    """Calculate duration between start date and current date in 'X year Y month' format."""
    try:
        # Parse the start date string
        if start_date_str == "Not found" or not start_date_str:
            return "Not found"

        # Try to parse the date with dateutil parser
        try:
            start_date = dateutil.parser.parse(start_date_str, fuzzy=True)
        except:
            # If parsing fails, try to extract month and year manually
            match = re.search(r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* (\d{4})',
                             start_date_str, re.IGNORECASE)
            if match:
                month_str = match.group(1)
                year_str = match.group(2)
                # Map abbreviated month to number
                month_map = {
                    'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4,
                    'may': 5, 'jun': 6, 'jul': 7, 'aug': 8,
                    'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
                }
                month = month_map.get(month_str.lower()[:3], 1)
                year = int(year_str)
                start_date = datetime(year, month, 1)
            else:
                return "Date format not recognized"

        # Calculate the difference between the start date and current date
        current_date = datetime.now()
        delta = relativedelta(current_date, start_date)

        # Format the result as "X year Y month"
        years = delta.years
        months = delta.months

        if years == 0:
            if months == 1:
                return f"{months} month"
            else:
                return f"{months} months"
        elif years == 1:
            if months == 0:
                return "1 year"
            elif months == 1:
                return "1 year 1 month"
            else:
                return f"1 year {months} months"
        else:
            if months == 0:
                return f"{years} years"
            elif months == 1:
                return f"{years} years 1 month"
            else:
                return f"{years} years {months} months"
    except Exception as e:
        return f"Error calculating duration: {str(e)}"

def extract_field(text, field_name):
    """Extract a specific field from text response when JSON parsing fails"""
    pattern = rf"{field_name}[:\s]+(.*?)(?:\n|$|,)"
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
        return match.group(1).strip()
    return "Not found"

def extract_cv_info(cv_text):
    """Use OpenAI API to extract structured information from CV text."""

    # Get current date for calculating work experience
    current_date = datetime.now()
    current_date_str = current_date.strftime("%Y %B")

    prompt = f"""
    Extract the following information from the CV text below.
    If you cannot find a particular piece of information, respond with "Not found" for that field.

    Information to extract:
    1. Name
    2. Last Education and university
    3. Number of total year experiences
    4. Present field of experience
    5. Overall expertise area
    6. Present organization designation
    7. Research experience (any research positions, publications, or projects)
    8. Achievements (awards, recognitions, significant accomplishments)
    9. Mobile number
    10. Email address
    11. Present organization name
    12. Working experience in present organization (start date in format 'Month YYYY', e.g. 'December 2022')

    Today is {current_date_str}.

    CV Text:
    {cv_text}

    Your response MUST be a valid JSON object with ONLY the following keys:
    {{
      "name": "extracted name",
      "last_education": "extracted education and university",
      "total_experience": "total number of experiences in all organizations",
      "present_field": "present field of experience",
      "overall_expertise_area": "areas of expertise or specialization",
      "present_organization_designation": "current job title or designation",
      "research_experience": "details of research experience if any",
      "achievements": "major achievements and awards if any",
      "mobile": "extracted mobile number",
      "email": "extracted email address",
      "present_organization_name": "name of current organization",
      "working_experience_in_present_organization": "start date in format 'Month YYYY'"
    }}

    Do not include any explanation, just return the JSON object.
    """

    try:
        # Use GPT-4o model for better extraction
        model = "gpt-4o"

        response = openai.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant that extracts structured information from CVs. Return only valid JSON."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3  # Lower temperature for more consistent results
        )

        # Extract and parse the JSON response
        result = response.choices[0].message.content

        # Try to find JSON in the response
        json_match = re.search(r'(\{[\s\S]*\})', result, re.DOTALL)
        if json_match:
            json_str = json_match.group(1)
            try:
                parsed_result = json.loads(json_str)

                # Calculate work experience duration
                start_date = parsed_result.get("working_experience_in_present_organization", "Not found")
                parsed_result["working_experience_in_year_in_present_organization"] = calculate_experience_duration(start_date)

                return parsed_result
            except json.JSONDecodeError:
                print(f"Failed to parse JSON from response. Attempting alternate extraction.")

        # If extraction failed, try to create a structured response manually
        try:
            # Create a standard response manually
            start_date = extract_field(result, "working_experience_in_present_organization")
            experience_duration = calculate_experience_duration(start_date)

            # Return in the specified order
            return {
                "name": extract_field(result, "name"),
                "last_education": extract_field(result, "last_education"),
                "overall_expertise_area": extract_field(result, "overall_expertise_area"),
                "present_organization_name": extract_field(result, "present_organization_name"),
                "present_organization_designation": extract_field(result, "present_organization_designation"),
                "working_experience_in_present_organization": start_date,
                "working_experience_in_year_in_present_organization": experience_duration,
                "total_experience": extract_field(result, "total_experience"),
                "present_field": extract_field(result, "present_field"),
                "research_experience": extract_field(result, "research_experience"),
                "achievements": extract_field(result, "achievements"),
                "mobile": extract_field(result, "mobile"),
                "email": extract_field(result, "email")
            }
        except Exception as e:
            print(f"Error creating structured response: {str(e)}")
            # Last resort, try direct JSON parsing
            parsed_result = json.loads(result)

            # Calculate work experience duration
            start_date = parsed_result.get("working_experience_in_present_organization", "Not found")
            parsed_result["working_experience_in_year_in_present_organization"] = calculate_experience_duration(start_date)

            return parsed_result

    except Exception as e:
        print(f"Error extracting information: {str(e)}")
        return {
            "name": "Error",
            "last_education": "Error",
            "total_experience": "Error",
            "present_field": "Error",
            "overall_expertise_area": "Error",
            "present_organization_designation": "Error",
            "research_experience": "Error",
            "achievements": "Error",
            "present_organization_name": "Error",
            "working_experience_in_present_organization": "Error",
            "working_experience_in_year_in_present_organization": "Error",
            "mobile": "Error",
            "email": "Error"
        }

# Function to find all PDF files in the directory structure
def find_pdf_files(root_path):
    pdf_files = []
    for root, _, files in os.walk(root_path):
        for file in files:
            if file.lower().endswith('.pdf'):
                # Get relative path components
                rel_path = os.path.relpath(root, root_path)
                if rel_path == '.':
                    subfolder = ""
                else:
                    subfolder = rel_path

                full_path = os.path.join(root, file)

                # Get file ID and create shareable link
                file_id = get_file_id(full_path)
                cv_link = create_shareable_link(file_id)

                pdf_files.append({
                    'full_path': full_path,
                    'filename': file,
                    'root_folder': root_path,
                    'subfolder': subfolder,
                    'cv_link': cv_link
                })
    return pdf_files

# Main execution
def main():
    print(f"Starting CV analysis from root path: {root_path}")

    # Find all PDF files
    pdf_files = find_pdf_files(root_path)
    print(f"Found {len(pdf_files)} PDF files to process")

    # Create a list to store results
    all_results = []

    # Process each PDF file
    for pdf_info in tqdm(pdf_files, desc="Processing CVs"):
        try:
            # Open the PDF file and extract text
            with open(pdf_info['full_path'], 'rb') as file:
                # Extract text from PDF
                cv_text = extract_text_from_pdf(file)

                # Extract structured information
                cv_info = extract_cv_info(cv_text)

                # Add file information
                cv_info["cv_links"] = pdf_info['filename']  # Will be displayed and renamed later
                cv_info["root_folder"] = pdf_info['root_folder']
                cv_info["subfolder"] = pdf_info['subfolder']
                cv_info["_cv_link"] = pdf_info['cv_link']  # Temporary field for processing

                # Add to results list
                all_results.append(cv_info)

                # Print progress
                print(f"Processed: {pdf_info['filename']}")
        except Exception as e:
            print(f"Error processing {pdf_info['filename']}: {str(e)}")
            # Add error entry
            error_info = {
                "name": "Error",
                "last_education": "Error",
                "total_experience": "Error",
                "present_field": "Error",
                "overall_expertise_area": "Error",
                "present_organization_designation": "Error",
                "research_experience": "Error",
                "achievements": "Error",
                "present_organization_name": "Error",
                "working_experience_in_present_organization": "Error",
                "working_experience_in_year_in_present_organization": "Error",
                "mobile": "Error",
                "email": "Error",
                "cv_links": pdf_info['filename'],
                "root_folder": pdf_info['root_folder'],
                "subfolder": pdf_info['subfolder'],
                "_cv_link": pdf_info['cv_link']
            }
            all_results.append(error_info)

    # Create a DataFrame from all results
    df = pd.DataFrame(all_results)

    # Define the desired column order
    column_order = [
        "name",
        "last_education",
        "overall_expertise_area",
        "present_organization_name",
        "present_organization_designation",
        "working_experience_in_present_organization",
        "working_experience_in_year_in_present_organization",
        "total_experience",
        "present_field",
        "research_experience",
        "achievements",
        "mobile",
        "email",
        "cv_links",  # Renamed from filename and will be the hyperlink
        "root_folder",
        "subfolder"
    ]

    # Reorder columns (only include columns that exist)
    existing_columns = [col for col in column_order if col in df.columns]
    extra_columns = [col for col in df.columns if col not in column_order]
    df = df[existing_columns + extra_columns]

    # Ensure all data is treated as strings to avoid conversion issues
    for column in df.columns:
        df[column] = df[column].astype(str)

    # Save the DataFrame to Excel
    output_path = os.path.join(root_path, "cv_analysis_results.xlsx")

    # Remove the temporary _cv_link column before saving
    if "_cv_link" in df.columns:
        df_save = df.drop("_cv_link", axis=1)
    else:
        df_save = df.copy()

    # First save using pandas to get the basic structure
    df_save.to_excel(output_path, index=False)

    # Now, modify the Excel file to create proper hyperlinks
    workbook = openpyxl.load_workbook(output_path)
    worksheet = workbook.active

    # Find the column indices for cv_links
    header_row = worksheet[1]
    cv_links_col_idx = None

    for idx, cell in enumerate(header_row, 1):
        if cell.value == "cv_links":
            cv_links_col_idx = idx

    # If the column exists, create hyperlinks
    if cv_links_col_idx:
        for row_idx in range(2, worksheet.max_row + 1):
            # Find the corresponding link from the original DataFrame
            if row_idx - 2 < len(df) and "_cv_link" in df.columns:
                file_link = df.iloc[row_idx-2]["_cv_link"]

                # Get the filename
                cv_links_cell = worksheet.cell(row=row_idx, column=cv_links_col_idx)
                filename = cv_links_cell.value

                if filename and file_link and file_link != "Link not available":
                    # Set the hyperlink
                    cv_links_cell.hyperlink = file_link
                    cv_links_cell.value = filename

                    # Format the cell as a hyperlink (blue and underlined)
                    cv_links_cell.font = Font(color="0000FF", underline="single")

    # Save the modified workbook to a final path for download
    final_output_path = os.path.join(root_path, "cv_analysis_results_with_links.xlsx")
    workbook.save(final_output_path)

    print(f"Analysis complete! Results saved to: {final_output_path} with clickable hyperlinks")

    # Automatically download the file
    files.download(final_output_path)

    return df_save
    return df

# Run the main function
if __name__ == "__main__":
    result_df = main()
    display(result_df)  # Display the results in the notebook

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Starting CV analysis from root path: /content/drive/MyDrive/JBC_HR_AI_ASSISTANT/22.04
Found 3 PDF files to process


Processing CVs:   0%|          | 0/3 [00:00<?, ?it/s]

Processed: Md Al Amin Tokder.pdf
Processed: resume-kaiser.pdf
Processed: resume_rizwan.pdf
Analysis complete! Results saved to: /content/drive/MyDrive/JBC_HR_AI_ASSISTANT/22.04/cv_analysis_results_with_links.xlsx with clickable hyperlinks


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,name,last_education,overall_expertise_area,present_organization_name,present_organization_designation,working_experience_in_present_organization,working_experience_in_year_in_present_organization,total_experience,present_field,research_experience,achievements,mobile,email,cv_links,root_folder,subfolder
0,MD AL AMIN TOKDER,"BSc in Computer Science and Engineering, Rajsh...","Machine Learning, Artificial Intelligence, Gen...",JB Connect Ltd,Machine Learning Engineer,January 2024,1 year 4 months,4,Machine Learning,Publications: Garbage Classification using a T...,"ICPC Dhaka Site-2023, Innovative Idea and Proj...",+8801750206042,alamintokdercse@gmail.com,Md Al Amin Tokder.pdf,/content/drive/MyDrive/JBC_HR_AI_ASSISTANT/22.04,
1,Kaiser Hamid Rabbi,"Master of Science, Otto-von-Guericke Universit...","Computer Vision, Natural Language Processing, ...",TigerIT,Senior Software Engineer,October 2019,5 years 7 months,6 years,Software Engineering,Author of 4 AI & Deep Learning courses on Udem...,Boosted number plate detection & tracking accu...,+8801644611605,kaiser.hamid.rabbi@gmail.com,resume-kaiser.pdf,/content/drive/MyDrive/JBC_HR_AI_ASSISTANT/22.04,
2,Syed Rizwan,"B.Sc in Computer Science & Engineering, Jashor...","Backend development, system design, Ruby on Ra...",JB Connect,Backend Software Engineer,January 2025,4 months,3 years,Backend Software Engineering,"Syed Rizwan, Md. Shazzad Ali Sobuj, and Mostaf...",Attended final round of Code Samurai 2022 inte...,01740404338,perizwansm@outlook.com,resume_rizwan.pdf,/content/drive/MyDrive/JBC_HR_AI_ASSISTANT/22.04,
